In [ ]:
!nvidia-smi

Thu Jul  4 15:28:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   70C    P8              19W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Πτυχιακή/Code/Enigma_ArgMining/train_dev_test_dataset/

/content/drive/MyDrive/Πτυχιακή/Code/Enigma_ArgMining/train_dev_test_dataset


In [ ]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q
!pip install neptune-client -q

# Importing the Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
import neptune.new as neptune

from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset)

from pandarallel import pandarallel
# Initialization
pandarallel.initialize(progress_bar = True)

[neptune] [warning] NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.


In [ ]:
run = neptune.init_run(
    project="enigma-replication/kpm",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmZjA5NzIzYi1mNzU4LTRiNDEtOGUxOC1iZWEyYjMzMGUwMTEifQ==",)  # your credentials

[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/enigma-replication/kpm/e/KPM-86


In [ ]:
SEED = 3
rn.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
device = 'cuda'

In [ ]:
path_dataset = '/content/drive/MyDrive/Πτυχιακή/Code/Enigma_ArgMining/train_dev_test_dataset/'
path_predictions_folder = '/content/drive/MyDrive/Πτυχιακή/Code/Enigma_ArgMining/train_dev_test_dataset/base_model_predictions_combined_dataset/'
save_model_folder = '/content/drive/MyDrive/Πτυχιακή/Code/Enigma_ArgMining/train_dev_test_dataset/base_model_files_combined_dataset/'

In [ ]:
max_len_arg = 55
max_len_kp = 32
max_len_topic = 12
#max_len_sent_1_sts = 58
#max_len_sent_2_sts = 65
#median_len_sent_1_sts = 9
#median_len_sent_2_sts = 9
max_len_input = 200
model_with_no_token_types =['distil_bert_gr']

### Function to make TensorDataset of the files

In [ ]:
import unicodedata
def strip_accents_and_lowercase(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').lower()

In [ ]:
def make_dataset(tokenizer, args, kps, topics, labels, max_len_input, model_with_no_token_types=model_with_no_token_types, model_name='distil_bert_gr'):

    all_input_ids = []
    all_token_type_ids = []
    all_attention_masks = []
    all_labels = []
    first_entry = True
    for arg, kp, topic, label in zip(args, kps, topics, labels):
        arg = strip_accents_and_lowercase(arg)
        kp = strip_accents_and_lowercase(kp)
        topic = strip_accents_and_lowercase(topic)

        arg = re.sub('[^a-zA-Zα-ωΑ-Ω]', ' ', arg)
        kp = re.sub('[^a-zA-Zα-ωΑ-Ω]', ' ', kp)
        topic = re.sub('[^a-zA-Zα-ωΑ-Ω]', ' ', topic)

        url = re.compile(r'https?://\S+|www\.\S+')
        arg = url.sub(r'', arg)
        kp = url.sub(r'', kp)
        topic = url.sub(r'', topic)

        html = re.compile(r'<.*?>')
        arg = html.sub(r'', arg)
        kp = html.sub(r'', kp)
        topic = html.sub(r'', topic)

        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)

        arg = emoji_pattern.sub(r'', arg)
        kp = emoji_pattern.sub(r'', kp)
        topic = emoji_pattern.sub(r'', topic)

        if model_name in model_with_no_token_types:
            encoded_input = tokenizer(kp+arg+topic,max_length = max_len_input, padding='max_length')
            all_input_ids.append(encoded_input['input_ids'])
            all_attention_masks.append(encoded_input['attention_mask'])
            # all_token_type_ids.append(encoded_input['token_type_ids'])
            all_labels.append(label)
        else:
            encoded_input = tokenizer(kp+arg+topic,max_length = max_len_input, padding='max_length')
            all_input_ids.append(encoded_input['input_ids'])
            all_attention_masks.append(encoded_input['attention_mask'])
            all_token_type_ids.append(encoded_input['token_type_ids'])
            all_labels.append(label)

        if first_entry:
            print("Preprocessed and encoded first data entry:")
            print(f"Argument: {arg}")
            print(f"Key Point: {kp}")
            print(f"Encoded input: {encoded_input}")
            tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'])
            print(tokens)
            first_entry = False

    if model_name in model_with_no_token_types:
        all_input_ids = torch.tensor(all_input_ids).squeeze()
        all_attention_masks = torch.tensor(all_attention_masks).squeeze()
        all_labels = torch.tensor(all_labels)

        dataset = TensorDataset(all_input_ids, all_attention_masks, all_labels)
    else:
        all_input_ids = torch.tensor(all_input_ids).squeeze()
        all_token_type_ids = torch.tensor(all_token_type_ids).squeeze()
        all_attention_masks = torch.tensor(all_attention_masks).squeeze()
        all_labels = torch.tensor(all_labels)

        dataset = TensorDataset(all_input_ids, all_token_type_ids, all_attention_masks, all_labels)

    return dataset


In [ ]:
def make_dataset_en(tokenizer, args, kps, topics, labels, max_len_input, model_with_no_token_types=model_with_no_token_types, model_name='distil_bert'):

    first_entry = True

    all_input_ids = []
    all_token_type_ids = []
    all_attention_masks = []
    all_labels = []

    for arg, kp, topic, label in zip(args, kps, topics, labels):

        arg = re.sub('[^a-zA-Z]', ' ', arg)
        kp = re.sub('[^a-zA-Z]', ' ', kp)
        topic= re.sub('[^a-zA-Z]', ' ', topic)

        url = re.compile(r'https?://\S+|www\.\S+')
        arg = url.sub(r'',arg)
        kp = url.sub(r'',kp)
        topic= url.sub(r'',topic)

        html=re.compile(r'<.*?>')
        arg = html.sub(r'',arg)
        kp = html.sub(r'',kp)
        topic= html.sub(r'',topic)

        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

        arg = emoji_pattern.sub(r'',arg)
        kp = emoji_pattern.sub(r'',kp)

        if model_name in model_with_no_token_types:

          encoded_input = tokenizer(kp+arg+topic,max_length = max_len_input, padding='max_length')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          #all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)

        else :

          encoded_input = tokenizer(kp+arg+topic,max_length = max_len_input, padding='max_length')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)
        if first_entry:
            print("Preprocessed and encoded first data entry:")
            print(f"Argument: {arg}")
            print(f"Key Point: {kp}")
            print(f"Encoded input: {encoded_input}")
            tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'])
            print(tokens)
            first_entry = False
    if model_name in model_with_no_token_types:
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_labels = torch.tensor(all_labels)

      dataset = TensorDataset(all_input_ids, all_attention_masks, all_labels)

    else :
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_token_type_ids = torch.tensor(all_token_type_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_labels = torch.tensor(all_labels)

      dataset = TensorDataset(all_input_ids,all_token_type_ids, all_attention_masks, all_labels)

    return dataset

## Loading the Dataset

In [ ]:
df_train = pd.read_csv(path_dataset+'train_translated.csv')
df_val = pd.read_csv(path_dataset+'validation_translated.csv')
df_test  = pd.read_csv(path_dataset+'final_human_translated_test.csv')

print(df_train.shape,df_val.shape,df_test.shape)

df_train.head()

(20635, 8) (3458, 8) (3923, 7)


,Unnamed: 0,arg_id,key_point_id,label,arg,key_point,topic,stance
0,0,arg_0_0,kp_0_0,0,Οι άνθρωποι φτάνουν στο όριο της ποιότητας ζωή...,Η υποβοηθούμενη αυτοκτονία δίνει αξιοπρέπεια σ...,Η υποβοηθούμενη αυτοκτονία θα πρέπει να είναι ...,-1
1,1,arg_0_121,kp_0_4,0,μια θεραπεία μπορεί να ανακαλυφθεί λίγο μετά τ...,Η υποβοηθούμενη αυτοκτονία επιτρέπει στους ανθ...,Η υποβοηθούμενη αυτοκτονία θα πρέπει να είναι ...,1
2,2,arg_0_121,kp_0_5,0,μια θεραπεία μπορεί να ανακαλυφθεί λίγο μετά τ...,Η υποβοηθούμενη αυτοκτονία είναι παρόμοια με τ...,Η υποβοηθούμενη αυτοκτονία θα πρέπει να είναι ...,1
3,3,arg_0_121,kp_0_6,1,μια θεραπεία μπορεί να ανακαλυφθεί λίγο μετά τ...,Η υποβοηθούμενη αυτοκτονία δεν πρέπει να επιτρ...,Η υποβοηθούμενη αυτοκτονία θα πρέπει να είναι ...,1
4,4,arg_0_121,kp_0_7,0,μια θεραπεία μπορεί να ανακαλυφθεί λίγο μετά τ...,Η υποβοηθούμενη αυτοκτονία παραβιάζει το ρόλο ...,Η υποβοηθούμενη αυτοκτονία θα πρέπει να είναι ...,1


In [ ]:
# Here I am concatenating train and dev set for training the model for creating test dataset predictions
df_train = pd.concat([df_train,df_val])
df_train.shape

(24093, 8)

types_of_models= model : tokenizer model_path
 - 'bert':  'bert-base-uncased'
 - 'roberta':  'roberta-base'
 - 'bart':  "facebook/bart-base"
 - 'distilbert': 'distilbert-base-uncased'
 - 'deberta': 'microsoft/deberta-base'
 - 'debertalarge': 'microsoft/deberta-large'
 - 'xlnet' : 'xlnet-base-cased'
 - 'xlnetlarge' : 'xlnet-large-cased'
 - 'xlmrobertalarge' : 'xlm-roberta-large'
 - 'bartlarge' : 'facebook/bart-large'
 - 'bertlarge':  'bert-large-uncased'
 - 'robertalarge':  'roberta-large'

In [ ]:
model_name ='distil_bert_gr'
model_path = 'EftychiaKarav/DistilGREEK-BERT'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path,do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_dataset = make_dataset(tokenizer, df_train['arg'], df_train['key_point'], df_train['topic'], df_train['label'], max_len_input, model_with_no_token_types, model_name=model_name)
val_dataset = make_dataset(tokenizer, df_val['arg'], df_val['key_point'], df_val['topic'], df_val['label'], max_len_input, model_with_no_token_types, model_name=model_name)
test_dataset = make_dataset(tokenizer, df_test['argument'], df_test['keypoint'], df_test['topic'], df_test['stance'], max_len_input, model_with_no_token_types, model_name=model_name)

Preprocessed and encoded first data entry:
Argument: οι ανθρωποι φτανουν στο οριο της ποιοτητας ζωης τους και θα πρεπει να ειναι σε θεση να τερματισουν τον πονο τους αυτο μπορει να γινει με ελαχιστο η καθολου πονο με βοηθεια και το ατομο ειναι σε θεση να πει αντιο 
Key Point: η υποβοηθουμενη αυτοκτονια δινει αξιοπρεπεια στο ατομο που θελει να την διαπραξει
Encoded input: {'input_ids': [101, 239, 34393, 19116, 823, 5369, 359, 1710, 354, 782, 348, 350, 28004, 483, 709, 6080, 359, 2457, 349, 1253, 636, 364, 344, 361, 396, 348, 357, 356, 481, 348, 15418, 413, 362, 2839, 364, 376, 395, 348, 502, 351, 2488, 239, 1588, 2839, 351, 1036, 344, 345, 1710, 357, 356, 481, 348, 959, 15089, 239, 34393, 19116, 361, 396, 348, 357, 11730, 11645, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Model Architecture

In [ ]:
# Use this Class only for Bert base and Bert large model

class Transformer(nn.Module):

    def __init__(self):
        super(Transformer, self).__init__()

        #Instantiating Pre trained model object
        self.model_layer = AutoModel.from_pretrained(model_path)

        #Layers
        # the first dense layer will have 768 neurons if base model is used and
        # 1024 neurons if large model is used

        self.dense_layer_1 = nn.Linear(768, 256)
        self.dropout = nn.Dropout(0.4)
        self.dense_layer_2 = nn.Linear(256, 128)
        self.dropout_2 = nn.Dropout(0.4)
        self.cls_layer = nn.Linear(128, 1, bias = True)
        self.sigmoid = nn.Sigmoid()

    def forward(self,input_ids, attention_masks, token_type_ids):

        pooled_output = self.model_layer(input_ids=input_ids, attention_mask=attention_masks,token_type_ids = token_type_ids).pooler_output

        x = self.dense_layer_1(pooled_output)
        x = self.dropout(x)
        x_1 = self.dense_layer_2(x)
        x_2 = self.dropout_2(x_1)

        logits = self.cls_layer(x_2)
        output = self.sigmoid(logits)

        return output

In [ ]:
# Use this Class for the rest of transformer models

class NonPoolerTransformer(nn.Module):

    def __init__(self):
        super(NonPoolerTransformer, self).__init__()

        #Instantiating Pre trained model object
        self.model_layer = AutoModel.from_pretrained(model_path)

        #Layers
        # the first dense layer will have 768 if base model is used and
        # 1024 if large model is used

        self.dense_layer_1 = nn.Linear(768, 256)
        self.dropout = nn.Dropout(0.4)
        self.dense_layer_2 = nn.Linear(256, 128)
        self.dropout_2 = nn.Dropout(0.2)
        self.cls_layer = nn.Linear(128, 1, bias = True)
        self.sigmoid = nn.Sigmoid()

    def forward(self,input_ids, attention_masks):

        hidden_state = self.model_layer(input_ids=input_ids, attention_mask=attention_masks)[0]
        pooled_output = hidden_state[:, 0]

        x = self.dense_layer_1(pooled_output)
        x = self.dropout(x)
        x_1 = self.dense_layer_2(x)
        x_2 = self.dropout_2(x_1)

        logits = self.cls_layer(x_2)
        output = self.sigmoid(logits)

        return output

In [ ]:
# comment the nonpoolertransformer line for bert model and the transformer model line for the rest of the transformer models
model = NonPoolerTransformer()
#model = Transformer()

In [ ]:
model = model.to(device)

In [ ]:
model

In [ ]:
BATCH_SIZE = 16
LEARNING_RATE = 1e-5
EPOCHS = 10
ACCUMULATION_STEPS = 2
DROPOUT = 0.4
gold_data_dir = path_dataset

In [ ]:
PARAMS = {'model_name': model_name,'model_path': model_path,'lr': LEARNING_RATE, 'epoch_nr': EPOCHS, 'batch_size': BATCH_SIZE, 'accumulation_steps': ACCUMULATION_STEPS,'dropout': DROPOUT}
run['details'] = PARAMS

*κείμενο σε πλάγια γραφή*### Evaluation functions

In [ ]:
def load_kpm_data(gold_data_dir, subset):

    arguments_file = os.path.join(gold_data_dir, f"arguments_translated_{subset}.csv")
    key_points_file = os.path.join(gold_data_dir, f"key_points_translated_{subset}.csv")
    labels_file = os.path.join(gold_data_dir, f"labels_{subset}.csv")

    arguments_df = pd.read_csv(arguments_file)
    key_points_df = pd.read_csv(key_points_file)
    labels_file_df = pd.read_csv(labels_file)

    return arguments_df, key_points_df, labels_file_df

In [ ]:
def get_predictions(predictions_file, labels_df, arg_df, kp_df):
    #print("\nֿ** loading predictions:")
    arg_df = arg_df[["arg_id", "topic", "stance"]]
    predictions_df = load_predictions(predictions_file, kp_df["key_point_id"].unique())

    #make sure each arg_id has a prediction
    predictions_df = pd.merge(arg_df, predictions_df, how="left", on="arg_id")

    #handle arguments with no matching key point
    predictions_df["key_point_id"] = predictions_df["key_point_id"].fillna("dummy_id")
    predictions_df["score"] = predictions_df["score"].fillna(0)

    #merge each argument with the gold labels
    merged_df = pd.merge(predictions_df, labels_df, how="left", on=["arg_id", "key_point_id"])

    merged_df.loc[merged_df['key_point_id'] == "dummy_id", 'label'] = 0
    merged_df["label_strict"] = merged_df["label"].fillna(0)
    merged_df["label_relaxed"] = merged_df["label"].fillna(1)

    return merged_df

In [ ]:
def load_predictions(predictions_dir, correct_kp_list):
    arg =[]
    kp = []
    scores = []
    invalid_keypoints = set()
    with open(predictions_dir, "r") as f_in:
        res = json.load(f_in)
        for arg_id, kps in res.items():
            valid_kps = {key: value for key, value in kps.items() if key in correct_kp_list}
            invalid = {key: value for key, value in kps.items() if key not in correct_kp_list}
            for invalid_kp, _ in invalid.items():
                if invalid_kp not in invalid_keypoints:
                    #print(f"key point {invalid_kp} doesn't appear in the key points file and will be ignored")
                    invalid_keypoints.add(invalid_kp)
            if valid_kps:
                best_kp = max(valid_kps.items(), key=lambda x: x[1])
                arg.append(arg_id)
                kp.append(best_kp[0])
                scores.append(best_kp[1])
        #print(f"\tloaded predictions for {len(arg)} arguments")

        return pd.DataFrame({"arg_id" : arg, "key_point_id": kp, "score": scores})

In [ ]:
def get_ap(df, label_column, top_percentile=0.5):
    top = int(len(df)*top_percentile)
    df = df.sort_values('score', ascending=False).head(top)
    # after selecting top percentile candidates, we set the score for the dummy kp to 1, to prevent it from increasing the precision.
    df.loc[df['key_point_id'] == "dummy_id", 'score'] = 0.99
    return average_precision_score(y_true=df[label_column], y_score=df["score"])

def calc_mean_average_precision(df, label_column):
    precisions = [get_ap(group, label_column) for _, group in df.groupby(["topic", "stance"])]
    return np.mean(precisions)

def evaluate_predictions(merged_df,name = 'train'):
    #print("\n** running evalution:")
    mAP_strict = calc_mean_average_precision(merged_df, "label_strict")
    mAP_relaxed = calc_mean_average_precision(merged_df, "label_relaxed")
    # below two lines are added for neptune results logging
    run[f"{name}/map"].log(mAP_strict)
    run[f"{name}/map_relaxed"].log(mAP_relaxed)

    print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

### Train and Predict Functions

In [ ]:
def evaluate_model(test_dataset,df, model,  model_name, mode = 'train'):

    save_predictions_name = model_name+ '__VAL_PREDS_'+ 'SEED_'+ str(SEED) + '_dense_layer' +'_epoc_'+ str(EPOCHS)+'_lr_'+ str(LEARNING_RATE)+'_b_s_'+ str(BATCH_SIZE) +'_accumulation_steps_'+ str(ACCUMULATION_STEPS) +'_input_type_kp_arg_topic_'

    y_preds = []
    val_losses = []
    criterion = nn.BCELoss()
    list_of_batch_losses = []

    if mode in ['train','val']:
        test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    with torch.no_grad():
        acc_epoch = []

        epoch_iterator = tqdm(test_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.eval()

            if model_name in model_with_no_token_types:
                b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
                ypred = model(b_input_ids, b_input_mask)
            else:
                b_input_ids,b_token_type, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
                ypred = model(b_input_ids, b_input_mask,b_token_type)

            b_labels_copy = torch.reshape(b_labels, (b_labels.shape[0], 1))
            loss_batch = criterion(ypred, b_labels_copy.float())
            list_of_batch_losses.append(loss_batch.detach().cpu().numpy())
            run["val/batch_loss"].log(np.mean(loss_batch.detach().cpu().numpy()))

            ypred = ypred.cpu().numpy()
            b_labels = batch[-1].cpu().detach().numpy()

            ypred = np.hstack(ypred)
            y_preds.append(ypred)

    epoch_loss = np.mean(list_of_batch_losses)
    val_losses.append(epoch_loss)
    run["val/epoch_loss"].log(epoch_loss)

    args = df['arg_id']
    kps = df['key_point_id']
    true_labels = df['label']
    topics = df['topic']
    stances = df['stance']
    all_preds = []

    for i in tqdm(range(len(y_preds))):
      for p in y_preds[i]:
        all_preds.append(p)

    print('Val evaluation....')

    pred_file = pd.DataFrame({"arg_id" : args, "key_point_id": kps, "score": all_preds})
    args = {}
    kps = {}

    for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
        args[arg] = {}

    for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
        args[arg][kp] = score

    with open(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', 'w') as fp:
        fp.write(json.dumps(args))
        fp.close()

    arg_df, kp_df, labels_df = load_kpm_data(path_dataset, subset="dev")
    merged_df = get_predictions(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', labels_df, arg_df, kp_df)

    evaluate_predictions(merged_df,name = 'val')

    return all_preds,true_labels, val_losses

In [ ]:
def train_and_evaluate(train_dataset,df,model, filepath, model_name, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, epochs = EPOCHS,accumulation_steps = ACCUMULATION_STEPS):

  train_losses = []
  val_losses = []

  save_model = model_name+ '_SEED_'+ str(SEED) +'_dense_layer' +'_epoc_'+ str(epochs)+'_lr_'+ str(learning_rate)+'_b_s_'+ str(batch_size ) +'_accumulation_steps_'+ str(accumulation_steps) +'_input_type_kp_arg_topic'
  save_predictions_name  = model_name+ '__TRAIN_PREDS_'+ 'SEED_'+ str(SEED) + '_dense_layer' +'_epoc_'+ str(epochs)+'_lr_'+ str(learning_rate)+'_b_s_'+ str(batch_size ) +'_accumulation_steps_'+ str(accumulation_steps) +'_input_type_kp_arg_topic'

  training_dataloader = DataLoader(train_dataset, batch_size )
  total_steps = len(training_dataloader) * epochs
  no_decay = ['bias', 'LayerNorm.weight']

  optimizer_grouped_parameters = [
                                  {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                  {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
                                  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps = 1e-8)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

  criterion = nn.BCELoss()

  model.zero_grad()

  for epoch_i in tqdm(range(epochs)):
    y_preds = []
    y_val = []
    list_of_batch_losses = []
    epoch_iterator = tqdm(training_dataloader, desc="Iteration")
    model.train()

    for step, batch in enumerate(epoch_iterator):
      if model_name in model_with_no_token_types:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        outputs = model(b_input_ids, b_input_mask)
      else:
        b_input_ids,b_token_type, b_input_mask,b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        outputs = model(b_input_ids, b_input_mask,b_token_type)

      b_labels = torch.reshape(b_labels, (b_labels.shape[0], 1))
      loss = criterion(outputs, b_labels.float())

      list_of_batch_losses.append(loss.detach().cpu().numpy())
      run["train/batch_loss"].log(np.mean(loss.detach().cpu().numpy()))

      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      ypred = outputs.detach().cpu().numpy()
      b_labels = batch[-1].cpu().detach().numpy()
      ypred = np.hstack(ypred)
      y_preds.append(ypred)

      if (step+1) % accumulation_steps == 0:
        optimizer.step()
        scheduler.step()
        model.zero_grad()


    epoch_loss = np.mean(list_of_batch_losses)
    train_losses.append(epoch_loss)
    run["train/epoch_loss"].log(epoch_loss)

    args = df['arg_id']
    kps = df['key_point_id']
    true_labels = df['label']
    topics = df['topic']
    stances = df['stance']
    all_preds = []

    for i in tqdm(range(len(y_preds))):
      for p in y_preds[i]:
        all_preds.append(p)

    print('Train evaluation....')

    pred_file = pd.DataFrame({"arg_id" : args, "key_point_id": kps, "score": all_preds})
    args = {}
    kps = {}

    for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
        args[arg] = {}

    for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
        args[arg][kp] = score

    with open(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', 'w') as fp:
        fp.write(json.dumps(args))
        fp.close()

    arg_df, kp_df, labels_df = load_kpm_data(path_dataset, subset="train")
    merged_df = get_predictions(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', labels_df, arg_df, kp_df)

    evaluate_predictions(merged_df,name = 'train')

    _,_, val_epoch_loss = evaluate_model(val_dataset,df_val, model,  model_name, mode = 'val')
    val_losses.append(val_epoch_loss)


  torch.save(model, save_model_folder +save_model+'.pt')
  run["model"].upload(save_model_folder +save_model+'.pt')

  print("Model is saved as : ",save_model)
  print("Use this to load the model")

  return save_model,train_losses, val_losses

In [ ]:
def predict_model(test_dataset,df, save_model,model_name):
  preds = []

  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
  save_predictions_name = "TEST_PREDS_"+ '_SEED_'+ str(SEED) + save_model

  model=torch.load(save_model_folder + save_model +'.pt')

  with torch.no_grad():

    epoch_iterator = tqdm(test_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      model.eval()

      if model_name in model_with_no_token_types:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        ypred = model(b_input_ids, b_input_mask)
      else:
        b_input_ids,b_token_type, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        ypred = model(b_input_ids, b_input_mask,b_token_type)

      ypred = ypred.cpu().numpy()
      ypred = np.hstack(ypred)

      preds.append(ypred)

  args = df['arg_id']
  kps = df['key_point_id']
  all_preds = []

  for i in tqdm(range(len(preds))):
    for p in preds[i]:
      all_preds.append(p)

  pred_file = pd.DataFrame({"arg_id" : args, "key_point_id": kps, "score": all_preds})

  args = {}
  kps = {}

  for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
    args[arg] = {}

  for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
    args[arg][kp] = score

  with open(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', 'w') as fp:
    fp.write(json.dumps(args))
    fp.close()

  print("The predictions are stored in the file : "+ path_predictions_folder  + save_predictions_name + '_' + 'predictions.p.')

  return path_predictions_folder + save_predictions_name + '_' + 'predictions.p.'

In [ ]:
def give_test_results(pred_file_path):
  print('The strict and relaxed scores on the test set predictions are: ')
  arg_df, kp_df, labels_df = load_kpm_data(gold_data_dir, subset="test")
  merged_df = get_predictions(pred_file_path, labels_df, arg_df, kp_df)
  evaluate_predictions(merged_df)

In [ ]:
save_model, train_losses, val_losses = train_and_evaluate(train_dataset,df_train, model, save_model_folder, model_name = model_name, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, epochs = EPOCHS, accumulation_steps = ACCUMULATION_STEPS)

  0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1506 [00:00<?, ?it/s]

  0%|          | 0/1506 [00:00<?, ?it/s]

Train evaluation....
mAP strict= 0.3169857607802598 ; mAP relaxed = 0.3169857607802598


Iteration:   0%|          | 0/217 [00:00<?, ?it/s]

...

  0%|          | 0/1506 [00:00<?, ?it/s]

Train evaluation....
mAP strict= 0.7033566975315716 ; mAP relaxed = 0.7033566975315716


Iteration:   0%|          | 0/217 [00:00<?, ?it/s]

  0%|          | 0/217 [00:00<?, ?it/s]

Val evaluation....
mAP strict= 0.839151737823165 ; mAP relaxed = 0.839151737823165
Model is saved as :  distil_bert_gr_SEED_3_dense_layer_epoc_10_lr_1e-05_b_s_16_accumulation_steps_2_input_type_kp_arg_topic
Use this to load the model


In [ ]:
test_preds_path = predict_model (test_dataset,df_test, save_model,model_name = model_name)

Iteration:   0%|          | 0/246 [00:00<?, ?it/s]

  0%|          | 0/246 [00:00<?, ?it/s]

The predictions are stored in the file : /content/drive/MyDrive/Πτυχιακή/Code/Enigma_ArgMining/train_dev_test_dataset/base_model_predictions_combined_dataset/TEST_PREDS__SEED_3distil_bert_gr_SEED_3_dense_layer_epoc_10_lr_1e-05_b_s_16_accumulation_steps_2_input_type_kp_arg_topic_predictions.p.


In [ ]:
give_test_results(test_preds_path)

The strict and relaxed scores on the test set predictions are: 
mAP strict= 0.5004736339174017 ; mAP relaxed = 0.5698558750437658
